In [8]:
# mpu6050_gyro.py
import tdwf
import time
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

# --- Registri principali (interi) ---
PWR_MGMT1     = 0x6B
PWR_MNGMT2    = 0x6C
WHO_AM_I      = 0x75
SMPLRT_DIV    = 0x19
CONFIG        = 0x1A
GYRO_CONFIG   = 0x1B
GYRO_XOUT_H   = 0x43  # burst read 6 bytes: XH XL YH YL ZH ZL
SAD           = 0x68

# --- Parametri giroscopio (FS_SEL e sensibilità) ---
# GYRO_CONFIG bits (FS_SEL):
# 0x00 -> ±250 °/s  -> 131.0 LSB/(°/s)
# 0x08 -> ±500 °/s  -> 65.5  LSB/(°/s)
# 0x10 -> ±1000 °/s -> 32.8  LSB/(°/s)
# 0x18 -> ±2000 °/s -> 16.4  LSB/(°/s)
# Qui uso ±250 °/s per miglior risoluzione (modifica se ti serve altro)
GYRO_CONFIG_VALUE = 0x00
GYRO_SENSITIVITY = 131.0  # LSB per °/s per register map for FS_SEL=0

def twos_comp(val, bits=16):
    """Two's complement conversion for `bits`-bit signed integer."""
    if val & (1 << (bits - 1)):
        val -= 1 << bits
    return val

def read_gyro(sht):
    """Burst-read 6 bytes from GYRO_XOUT_H and return (gx, gy, gz) in °/s."""
    sht.writeread([GYRO_XOUT_H], 6)     # read XH XL YH YL ZH ZL
    hx, lx, hy, ly, hz, lz = sht.vals[0:6]
    raw_x = (hx << 8) | lx
    raw_y = (hy << 8) | ly
    raw_z = (hz << 8) | lz
    raw_x = twos_comp(raw_x, 16)
    raw_y = twos_comp(raw_y, 16)
    raw_z = twos_comp(raw_z, 16)
    gx = raw_x / GYRO_SENSITIVITY
    gy = raw_y / GYRO_SENSITIVITY
    gz = raw_z / GYRO_SENSITIVITY
    return gx, gy, gz, raw_x, raw_y, raw_z

def main():
    ad2 = tdwf.AD2()
    try:
        ad2.vdd = 3.3
        ad2.power(True)
        i2c = tdwf.I2Cbus(ad2.hdwf)
        devs = i2c.scan()
        print("I2C devices:", [hex(d) for d in devs])

        sht = tdwf.I2Cdevice(ad2.hdwf, SAD)

        # Reset / Wake / Configurazione base
        sht.write([PWR_MGMT1, 0x80])   # reset device
        time.sleep(0.5)
        sht.write([PWR_MGMT1, 0x00])   # wake up, set clock source
        time.sleep(0.05)
        sht.write([PWR_MNGMT2, 0x00])  # enable accel & gyro (temp indipendente)
        time.sleep(0.01)
        sht.write([CONFIG, 0x03])      # DLPF (opzionale)
        time.sleep(0.01)
        sht.write([SMPLRT_DIV, 0x04])  # sample rate divider (opzionale)
        time.sleep(0.01)

        # Imposta full scale del giroscopio
        sht.write([GYRO_CONFIG, GYRO_CONFIG_VALUE])
        time.sleep(0.01)

        # Verifica WHO_AM_I
        sht.writeread([WHO_AM_I], 1)
        if sht.vals[0] != 0x68:
            print("WHO_AM_I mismatch: 0x%02x" % sht.vals[0])
            return

        # Setup plot (solo velocità angolare per i tre assi)
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8, 8), sharex=True)
        plt.subplots_adjust(hspace=0.4)
        start = time.time()
        times, GX, GY, GZ = [], [], [], []
        ax1.set_ylabel("Gx [°/s]"); ax1.grid(True)
        ax2.set_ylabel("Gy [°/s]"); ax2.grid(True)
        ax3.set_ylabel("Gz [°/s]"); ax3.set_xlabel("Time [s]"); ax3.grid(True)
        line_gx, = ax1.plot([], [], marker='.', linestyle='-')
        line_gy, = ax2.plot([], [], marker='.', linestyle='-')
        line_gz, = ax3.plot([], [], marker='.', linestyle='-')

        running = True
        def on_key(event):
            nonlocal running
            if event.key == 'escape':
                running = False
        fig.canvas.mpl_connect("key_press_event", on_key)
        plt.show(block=False)

        print("Plotting gyroscope (°/s). Premere ESC nella finestra per fermare.")

        while running:
            gx, gy, gz, rx, ry, rz = read_gyro(sht)
            t = time.time() - start
            times.append(t)
            GX.append(gx); GY.append(gy); GZ.append(gz)

            # stampa rapida su console
            print(f"{t:6.2f}s  Gx={gx:7.2f}  Gy={gy:7.2f}  Gz={gz:7.2f}  (raw: {rx}, {ry}, {rz})")

            # aggiorna grafico
            line_gx.set_data(times, GX)
            line_gy.set_data(times, GY)
            line_gz.set_data(times, GZ)

            for ax in (ax1, ax2, ax3):
                ax.relim()
                ax.autoscale_view()

            fig.canvas.draw()
            fig.canvas.flush_events()

            time.sleep(0.02)  # loop ~50 Hz (dipende anche da SMPLRT_DIV / DLPF)

    finally:
        print("Cleaning up hardware...")
        try:
            plt.close('all')
        except:
            pass
        ad2.power(False)
        ad2.close()
        print("AD2 Power Off. Process safely terminated.")

if __name__ == "__main__":
    main()


Dispositivo #1 [SN:210321B5D136, hdwf=1] connesso!
Configurazione #1
Bus I2C pronto...
I2C devices: ['0x68']
Plotting gyroscope (°/s). Premere ESC nella finestra per fermare.
  0.05s  Gx=   1.66  Gy=   0.17  Gz=  -1.66  (raw: 217, 22, -218)
  0.56s  Gx=   1.50  Gy=   0.14  Gz=  -0.79  (raw: 196, 18, -103)
  0.71s  Gx=   1.48  Gy=   0.00  Gz=  -1.18  (raw: 194, 0, -155)
  0.83s  Gx=   1.53  Gy=   0.02  Gz=  -1.21  (raw: 200, 2, -158)
  0.96s  Gx=   1.53  Gy=  -0.02  Gz=  -1.12  (raw: 201, -3, -147)
  1.08s  Gx=   1.53  Gy=   0.03  Gz=  -1.11  (raw: 200, 4, -145)
  1.19s  Gx=   1.57  Gy=   0.04  Gz=  -1.00  (raw: 206, 5, -131)
  1.30s  Gx=   1.53  Gy=   0.08  Gz=  -1.00  (raw: 200, 10, -131)
  1.42s  Gx=   1.54  Gy=   0.07  Gz=  -1.03  (raw: 202, 9, -135)
  1.54s  Gx=   1.53  Gy=   0.04  Gz=  -0.99  (raw: 201, 5, -130)
  1.66s  Gx=   1.54  Gy=   0.02  Gz=  -1.03  (raw: 202, 2, -135)
  1.78s  Gx=   1.53  Gy=   0.04  Gz=  -1.14  (raw: 201, 5, -149)
  1.90s  Gx=   1.50  Gy=   0.06  Gz=  -0.

KeyboardInterrupt: 